# kakuro

1. Each cell can contain numbers from 1 through 9
2. The clues in the black cells tells the sum of the numbers next to that clue. (on the right or down)
3. The numbers in consecutive white cells must be unique.

-----

1. 每个空格只能包含从1到9的数字；
2. 黑色有数字的空格，斜线上的数字表示横向的连续空格里数字的和，斜线下的数字表示纵向的连续空格里数字的和；
3. 连续空格内的数字必须是不重复的。

------

## Input Rules 

```
16 16
- - - - - - - - - 6\ 24\ - 11\ 7\ - -
- - 41\ 17\ - - - - \11 0 0 \3 0 0 10\ 12\
- \17 0 0 - 12\ 7\ - 11\9 0 0 36\11 0 0 0 0
- 3\11 0 0 \15 0 0 16\25 0 0 0 0 0 5\8 0 0
\10 0 0 - \19 0 0 0 0 - 14\16 0 0 0 0 0
\7 0 0 7\ - - 10\3 0 0 \14 0 0 \6 0 0 -
- \5 0 0 18\ 30\9 0 0 - \6 0 0 21\ - 43\ 11\
- 10\25 0 0 0 0 0 - - - \7 0 0 \16 0 0
\16 0 0 \3 0 0 - - - - 9\16 0 0 7\3 0 0
\6 0 0 \16 0 0 10\ - - 24\31 0 0 0 0 0 -
- - 22\ 3\ \3 0 0 - 6\17 0 0 - \4 0 0 17\
- 24\5 0 0 20\13 0 0 \11 0 0 4\ 14\ - \15 0 0
\26 0 0 0 0 0 19\ 12\20 0 0 0 0 - 3\13 0 0
\11 0 0 8\23 0 0 0 0 0 \7 0 0 \9 0 0 -
\19 0 0 0 0 \17 0 0 - - - - \6 0 0 -
- - \15 0 0 \6 0 0 - - - - - - - -
```

第一行的两个数字m, n，表示网格的尺寸：高和宽；

接下来一共有 m  行，每一行代表网格的一行;

每一行一共 n 个数据，用空格隔开，有三种情况：

1. 如果是无法填数字，并且没有数字提示的，为`-`，

2. 如果是无法填数字，但是有数字标记的，用 `\` 对数字进行分割， `\` 前的数字是向下的连续空格数字和， `\` 后的数字是向右的连续空格数字和；

3. 如果是需要填数字的，那么标记为 0. 



![](../assets/figures/Kakuro1.png)

In [17]:
def readGrid(path):
    with open(f"../assets/data/Kakuro/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

In [25]:
from ortools.sat.python import cp_model as cp

class KakuroSolver:
    
    def __init__(self, X, Y, grid):
        self.X = X
        self.Y = Y
        self.grid = grid
        self.x = {}
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        for i in range(self.X):
            for j in range(self.Y):
                if self.grid[i][j] == "0":
                    self.x[i, j] = self.model.NewIntVar(1, 9, f'x[{i}, {j}]')

    def addConstrs(self):
        for i in range(self.X ):
            for j in range(self.Y ):
                if self.grid[i][j] == "0" or self.grid[i][j] == "-":
                    continue
                numbers = self.grid[i][j].split("\\")
                down_sum , up_sum = numbers[0], numbers[1]
                if len(down_sum) > 0:
                    tmp_x = i + 1
                    down_arr = []
                    while tmp_x < self.X and self.grid[tmp_x][j] == "0":
                        down_arr.append(self.x[tmp_x, j])
                        tmp_x += 1
                    self.model.AddAllDifferent(down_arr)
                    self.model.Add(sum(down_arr) == int(down_sum))
                
                if len(up_sum) > 0:
                    tmp_y = j + 1
                    up_arr = []
                    while tmp_y < self.Y and self.grid[i][tmp_y] == "0":
                        up_arr.append(self.x[i, tmp_y])
                        tmp_y += 1
                    self.model.AddAllDifferent(up_arr)
                    self.model.Add(sum(up_arr) == int(up_sum))
                    
    def printgrid(self):
        for i in range(self.X ):
            for j in range(self.Y ):
                if self.grid[i][j] != "0":
                    print("#", end = " ")
                else:
                    print(self.solver.Value(self.x[i, j]), end=" ")
            print()
        print()
        
        
    def solve(self):
        self.solver.Solve(self.model)
        print(self.solver.ResponseStats())
        self.printgrid()
        
    
if __name__ == "__main__":
    x, y, grid = readGrid("problems/426_12x14")
    # x, y, grid = readGrid("15x15_1")
    KakuroSolverTest = KakuroSolver(x, y, grid)
    KakuroSolverTest.addConstrs()
    KakuroSolverTest.solve()
        

CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.036275
usertime: 0.036275
deterministic_time: 0.000697299
gap_integral: 0
solution_fingerprint: 0x60bce7ca2d8fc15c

# # # # # # # # # # # # # # 
# # 8 4 # # 8 9 6 # # 3 1 # 
# 1 6 3 # 6 9 7 2 8 # 1 2 3 
# 5 9 6 7 8 # # # 5 3 2 4 1 
# # # 8 9 7 6 # 6 9 4 5 # # 
# 2 3 1 # 9 8 6 3 7 # 4 3 2 
# 3 1 # # # 4 2 1 # # # 1 4 
# 5 4 1 # 4 5 3 2 1 # 3 2 1 
# # # 3 5 1 2 # 4 2 3 1 # # 
# 3 1 6 4 2 # # # 4 1 6 3 2 
# 8 7 9 # 6 8 9 7 5 # 4 2 1 
# # 2 5 # # 4 1 2 # # 2 1 # 

